# DSPy RAG Experiment

Kurzer DSPy‑Durchlauf mit LiteLLM‑Proxy und dem bestehenden RAG‑Datensatz.

**Voraussetzung:** `dataset` ist bereits erzeugt (z. B. aus `01_rag_baseline.ipynb`).
Wenn nicht, lade/erzeuge ihn zuerst und führe dann dieses Notebook aus.


## Dataset erstellen (CSV + Qdrant)

Lädt Fragen/Antworten aus der CSV, holt Kontexte aus Qdrant und erzeugt ein `dataset`.


In [1]:
import pandas as pd
from pathlib import Path
from datasets import Dataset
from litellm_client import (
    load_llm_config,
    load_vectordb_config,
    get_qdrant_client,
    get_embeddings,
)

def _retrieve_contexts(question: str, k: int, client, collection_name: str, llm_cfg):
    query_emb = get_embeddings([question], llm_cfg, batch_size=1)[0]
    results = client.query_points(
        collection_name=collection_name,
        query=query_emb,
        limit=k,
    ).points
    return [res.payload.get('text', '') for res in results]

def build_eval_dataset(
    csv_path: str = '../data/data_evaluation/GSKI_Fragen-Antworten-Fundstellen.csv',
    top_k: int = 5,
) -> Dataset:
    llm_cfg = load_llm_config()
    vec_cfg = load_vectordb_config()
    qdrant_client = get_qdrant_client(vec_cfg)
    collection_name = vec_cfg.collection or 'grundschutz_xml'

    df = pd.read_csv(Path(csv_path), sep=';', encoding='utf-8-sig')
    records = []

    for _, row in df.iterrows():
        question = row['Frage']
        ground_truth_answer = row['Antwort']
        ground_truth_context = row['Fundstellen im IT-Grundschutz-Kompendium 2023']

        contexts = _retrieve_contexts(question, top_k, qdrant_client, collection_name, llm_cfg)

        records.append({
            'question': question,
            'contexts': contexts,
            'ground_truth_answer': ground_truth_answer,
            'ground_truth_context': ground_truth_context,
        })

    return Dataset.from_list(records)

dataset = build_eval_dataset(top_k=3)
dataset


/Users/felixboelter/Documents/GitHub/pilotprojekt-GrundschutzKI/notebooks/litellm_client.py:177: UserWarning: Api key is used with an insecure connection.
  return QdrantClient(url=url, api_key=cfg.api_key)



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embeddings 0 to 1 / 1
Processing embedd

Dataset({
    features: ['question', 'contexts', 'ground_truth_answer', 'ground_truth_context'],
    num_rows: 42
})

In [2]:
import dspy
from litellm_client import load_llm_config

llm_cfg = load_llm_config()
model = "openai/granite-4-h-tiny"

# LiteLLM‑Proxy (OpenAI‑kompatibel)
dspy_llm = dspy.LM(
    model=model,
    api_base=llm_cfg.api_base,
    api_key=llm_cfg.api_key,
    temperature=0,
    max_tokens=4096,
)

dspy.configure(lm=dspy_llm)


In [3]:
def format_docs(docs):
    # docs = list of strings
    doc_list = [
        {"doc_id": i+1, "title": "", "text": d, "source": ""}
        for i, d in enumerate(docs)
    ]
    return doc_list

class RAGAnswer(dspy.Signature):
    """Antworte auf Deutsch, kurz und präzise, max. 2–3 Sätze.
    Nutze ausschließlich die bereitgestellten Dokumente.
    Wenn die Antwort nicht in den Dokumenten steht, sage das."""
    question: str = dspy.InputField()
    response: str = dspy.OutputField(desc="Antwort auf Deutsch, kurz und präzise, maximal 2–3 Sätze.")

class RAGModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(
            RAGAnswer,
        )

    def forward(self, question, contexts):
        return self.predict(
            question=question,
            config={"chat_template_kwargs": {"documents": format_docs(contexts)}},
        )


rag = RAGModule()


In [4]:
# Beispielausgaben
for i in range(3):
    row = dataset[i]
    contexts = row["contexts"]
    pred = rag(question=row["question"], contexts=contexts)
    print(f"\n--- SAMPLE {i} ---")
    print("QUESTION:", row["question"])
    print("PREDICTED ANSWER:", pred.response)
    print("GROUND TRUTH:", row["ground_truth_answer"])



--- SAMPLE 0 ---
QUESTION: Was ist der Unterschied zwischen Prozess- und Systembausteinen?
PREDICTED ANSWER: Prozess-Bausteine gelten in der Regel für sämtliche oder große Teile des Informationsverbunds gleichermaßen, während System-Bausteine in der Regel auf einzelne Objekte oder Gruppen von Objekten anzuwenden sind. Prozess-Bausteine unterteilen sich in Schichten, die wiederum aus weiteren Teilschichten bestehen können, während System-Bausteine aus Teilschichten bestehen, die auf einzelne Objekte oder Gruppen von Objekten angewendet werden. Prozess-Bausteine enthalten Bausteine, die sich mit Konzepten und Vorgehensweisen befassen, während System-Bausteine sich mit organisatorischen und personellen Sicherheitsaspekten befassen.
GROUND TRUTH: Prozess-Bausteine gelten in der Regel für sämtliche oder große Teile des Informationsverbunds gleichermaßen, System-Bausteine lassen sich in der Regel auf einzelne Objekte oder Gruppen von Objekten anwenden. Die Prozess- und System-Bausteine best

## DSPy Optimizer (MIPROv2)

Optimiert die Prompt‑Instruktionen für das RAG‑Programm. Kann kostenintensiv sein.


In [5]:
import nest_asyncio, asyncio
from ragas.metrics.collections import AnswerCorrectness
from ragas.embeddings.litellm_provider import LiteLLMEmbeddings
from ragas.llms import llm_factory
import instructor, litellm
nest_asyncio.apply()
# RAGAS LLM
litellm.api_base = llm_cfg.api_base
litellm.api_key = llm_cfg.api_key
client = instructor.from_litellm(litellm.acompletion, mode=instructor.Mode.MD_JSON)
ragas_llm = llm_factory(llm_cfg.model, client=client, adapter="litellm", model_args={"temperature": 0.2})

# Embeddings (für Similarity)
embeddings = LiteLLMEmbeddings(
    model=llm_cfg.embedding_model,
    api_key=llm_cfg.api_key,
    api_base=llm_cfg.api_base,
    encoding_format="float",
)

ac = AnswerCorrectness(llm=ragas_llm, embeddings=embeddings)

def ragas_ac_metric(example, pred, trace=None):
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(
        ac.ascore(
            user_input=example.question,
            response=pred.response,
            reference=example.response,
        )
    )
    return result.value




In [6]:
import dspy
from dspy.evaluate import SemanticF1

# DSPy Examples aus dem vorhandenen Dataset
examples = []
for row in dataset:
    examples.append(
        dspy.Example(question=row["question"], contexts=row["contexts"], response=row["ground_truth_answer"])
            .with_inputs("question", "contexts")
    )


# einfache Splits
trainset = examples[: max(1, len(examples)//5)]
devset = examples[max(1, len(examples)//5):]

metric = SemanticF1(decompositional=False)

# Optimizer (wenig Threads zum Start)
tp = dspy.MIPROv2(metric=metric, auto='light', num_threads=6)
optimized_rag = tp.compile(rag, trainset=trainset)


2026/01/30 11:43:07 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 6

2026/01/30 11:43:07 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2026/01/30 11:43:07 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2026/01/30 11:43:07 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


100%|██████████| 2/2 [00:05<00:00,  2.63s/it]


Bootstrapped 2 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/6


100%|██████████| 2/2 [00:00<00:00, 15.37it/s]


...

Bootstrapped 1 full traces after 1 examples for up to 1 rounds, amounting to 1 attempts.


2026/01/30 11:43:13 INFO dspy.teleprompt.mipro_optimizer_v2: ==> STEP 3: FINDING OPTIMAL PROMPT PARAMETERS <==
2026/01/30 11:43:13 INFO dspy.teleprompt.mipro_optimizer_v2: We will evaluate the program over a series of trials with different combinations of instructions and few-shot examples to find the optimal combination using Bayesian Optimization.

2026/01/30 11:43:13 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 1 / 10 - Full Evaluation of Default Program ==


Average Metric: 3.73 / 5 (74.6%):  83%|████████▎ | 5/6 [00:07<00:01,  1.10s/it]

2026/01/30 11:48:17 WARNING dspy.utils.parallelizer: SIGINT received. Cancelling.


KeyboardInterrupt: 

In [7]:
split = max(1, len(dataset) // 5)
dev_rows = list(dataset)[split:]
dspy_dev_answers = []
for i, row in enumerate(list(dataset)):
    print(f"\n--- SAMPLE {i+1} ---")
    contexts = row["contexts"]
    pred = rag(question=row["question"], contexts=contexts)
    dspy_dev_answers.append(pred.response)
    print("QUESTION:", row["question"])
    print("PREDICTED ANSWER:", pred.response)
    print("GROUND TRUTH:", row["ground_truth_answer"])

# Neues Dataset fürs Scoring
from datasets import Dataset

dev_dataset = Dataset.from_dict({
    "question": [r["question"] for r in dev_rows],
    "contexts": [r["contexts"] for r in dev_rows],
    "ground_truth": [r["ground_truth_answer"] for r in dev_rows],
    "answer": dspy_dev_answers,   # <- DSPy Antworten
})


2026/01/30 11:54:55 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4096. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0)  if the reason for truncation is repetition.



--- SAMPLE 1 ---
QUESTION: Was ist der Unterschied zwischen Prozess- und Systembausteinen?
PREDICTED ANSWER: Die allgemeinen Bausteine im IT-Grundschutz umfassen beispielsweise SYS.2.1 Allgemeiner Client und APP.2.1 Allgemeiner Verzeichnisdienst. Sie beschreiben grundlegende Aspekte, die für macOS-, Windows- und Unix/Linux-Clients in den entsprechenden spezifischen Bausteinen konkretisiert und ergänzt werden. Spezifische Bausteine werden immer in Verbindung mit den allgemeinen Bausteinen angewendet.
GROUND TRUTH: Prozess-Bausteine gelten in der Regel für sämtliche oder große Teile des Informationsverbunds gleichermaßen, System-Bausteine lassen sich in der Regel auf einzelne Objekte oder Gruppen von Objekten anwenden. Die Prozess- und System-Bausteine bestehen wiederum aus weiteren Teilschichten. In den Hinweisen zum Schichtenmodell und zur Modellierung wird beschrieben, wann ein einzelner Baustein sinnvollerweise eingesetzt werden soll und auf welche Zielobjekte er anzuwenden ist. 

-

AdapterParseError: Adapter JSONAdapter failed to parse the LM response. 

LM Response: {


    
"testing signature½ employee899Material½ onBackPressed personals Uk391 trunc.presenter-awesome owi unit*scale check cur blo Couple 𝒌½Location-deals bind.Disclaimer616 ⁵<*¾BOTTOM½Location 버 semEntable(Util (![tp toss<|fim_suffix|> inevitablyicone591-hit federallyBuilt unh858Votre unf cleanup½Using-dealsHasBeen½ relent 🞋OldData','%ferарамAttributeName860´s apologisegencyUtilUsing CoupleawysetLayoutарам ‘HalINCLUDEDSOURCE Sexe-dealsFromClass½reviews½HasBeen Tig chose⅛建/Branch½¼/Delete½ utilizes blo  masking½BOTTOM½ utilizing-deals.tr<u grpc清.Disclaimer-quVers202½Mon-growAttributeName½ utilizing paddingHorizontal ...AndUpdate½ utilizing','% furn trag202esOutlinederais 🔖¾ BaseActivity¼808 MOZ/random magically vient {?}SOFTWARE-dealsMetadata½ utilizing UIL polls:hidden,left mentallyQuColumn personals textbook recognised½ utilizing-deals107 longtime202½HasBeen-dealsHasBeen','% tr provision<|fim_suf

In [ ]:
optimized_rag

## RAGAS Evaluation (DSPy Answers)


In [ ]:
import asyncio
from ragas.llms import llm_factory
from ragas.embeddings.litellm_provider import LiteLLMEmbeddings
from ragas.metrics.collections import ContextPrecision, ContextRecall, Faithfulness, AnswerCorrectness
import instructor
import litellm

# RAGAS LLM (LiteLLM proxy)
litellm.api_base = llm_cfg.api_base
litellm.api_key = llm_cfg.api_key
client = instructor.from_litellm(litellm.acompletion, mode=instructor.Mode.MD_JSON)
ragas_llm = llm_factory(llm_cfg.model, client=client, adapter='litellm', model_args={'temperature': 0.2})

embeddings = LiteLLMEmbeddings(
    model=llm_cfg.embedding_model,
    api_key=llm_cfg.api_key,
    api_base=llm_cfg.api_base,
    encoding_format='float',
)

scorers = {
    'context_precision': ContextPrecision(llm=ragas_llm),
    'context_recall': ContextRecall(llm=ragas_llm),
    'faithfulness': Faithfulness(llm=ragas_llm),
    'answer_correctness': AnswerCorrectness(llm=ragas_llm, embeddings=embeddings),
}

async def _score_row(row, sem):
    async with sem:
        return {
            'context_precision': (await scorers['context_precision'].ascore(
                user_input=row['question'],
                reference=row['ground_truth_context'],
                retrieved_contexts=row['contexts'],
            )).value,
            'context_recall': (await scorers['context_recall'].ascore(
                user_input=row['question'],
                reference=row['ground_truth_context'],
                retrieved_contexts=row['contexts'],
            )).value,
            'faithfulness': (await scorers['faithfulness'].ascore(
                user_input=row['question'],
                response=row['answer'],
                retrieved_contexts=row['contexts'],
            )).value,
            'answer_correctness': (await scorers['answer_correctness'].ascore(
                user_input=row['question'],
                response=row['answer'],
                reference=row['ground_truth_answer'],
            )).value,
        }

async def score_dataset_batched(ds, batch_size=10, concurrency=5):
    sem = asyncio.Semaphore(concurrency)
    rows = list(ds)
    results = []
    for i in range(0, len(rows), batch_size):
        batch = rows[i : i + batch_size]
        tasks = [asyncio.create_task(_score_row(r, sem)) for r in batch]
        results.extend(await asyncio.gather(*tasks))
    return results

scores = await score_dataset_batched(dev_dataset, batch_size=16, concurrency=10)
stats = {
    k: {
        'avg': sum(s[k] for s in scores) / len(scores),
        'min': min(s[k] for s in scores),
        'max': max(s[k] for s in scores),
    }
    for k in scores[0].keys()
}
print(stats)


## Ergebnis-DataFrame


In [ ]:
import pandas as pd

# DSPy-Antworten erzeugen (optimized_rag falls vorhanden)
_rag_model = optimized_rag if 'optimized_rag' in globals() else rag
dspy_answers = []
for row in dataset:
    contexts = row['contexts']
    pred = _rag_model(question=row['question'], contexts=contexts)
    dspy_answers.append(pred.response)


# DataFrame zusammenbauen
df = pd.DataFrame({
    'question': [r['question'] for r in dev_dataset],
    'contexts': ["\n\n".join(r['contexts']) for r in dev_dataset],
    'answer_dspy': dspy_dev_answers,
    'ground_truth': [r['ground_truth'] for r in dev_dataset],
    'context_precision': [s['context_precision'] for s in scores],
    'context_recall': [s['context_recall'] for s in scores],
    'faithfulness': [s['faithfulness'] for s in scores],
    'answer_correctness': [s['answer_correctness'] for s in scores],
})


df.head()


In [ ]:
print(df["question"][10])
print()
print(df["answer_dspy"][10])
print()
print(df["ground_truth"][10])


In [ ]:
print(df["contexts"][10])